In [21]:
import numpy as np
import pandas as pd
import random

Gaussian Elimination

![title](img_vsc/img01.png)

![title](img_vsc/img02.png)

In [3]:
def gaussian_elimination(A, B):
    
    n = A.shape[0]
    
    C = np.hstack((A, B.reshape(n, 1)))

    for i in range(n):
        for j in range(i + 1, n):
            ratio = C[j][i] / C[i][i]
            C[j] = C[j] - ratio * C[i]

    X = np.zeros(n)

    X[n - 1] = C[n - 1][n] / C[n - 1][n - 1] # We now have n + 1 columns

    for i  in range(n - 2 , -1, -1):
        X[i] = (C[i][n] - sum(X[i+1:n]*C[i][i+1:n])) / C[i][i]
        
    return X

In [52]:
class GaussTest():
    
    def __init__(self, get_matrix_function):
        self.get_matrix = get_matrix_function

    def random_vector(self, n):
        
        random.seed(555)
        
        V = []
        
        for _ in range(n):
            V.append( -1 if random.random() < 0.5 else 1)
            
        return V
        
    def solve(self, n):
        
        A = self.get_matrix(n)   
        X = self.random_vector(n)
        
        A = np.array(A).reshape(n, n)
        X = np.array(X).reshape(n)
        
        B = A @ X
        
        X_approx = gaussian_elimination(A, B)
        
        return X, X_approx

In [53]:
def zad1_matrix(n):
    
    A = []
        
    for i in range(1, n + 1):
        for j in range(1, n + 1):
                
            if i == 1:
                A.append(1)
            else:
                A.append(1 / (i + j - 1)) 
                    
    return A

In [54]:
zad1 = GaussTest(zad1_matrix)

X, X_approx = zad1.solve(5)

print(X, X_approx)

[-1 -1  1  1 -1] [-1. -1.  1.  1. -1.]


In [92]:
def zad2_matrix(n):
    
    A = [[0 for j in range(1, n + 1)] for i in range(1, n + 1)]
    
    for i in range(1, n + 1):
        for j in range(1, n + 1):
            
            if j >= i:
                A[i - 1][j - 1] = 2 * i / j
            else:
                A[i - 1][j - 1] = A[j - 1][i - 1]
                
    return A

In [93]:
zad2 = GaussTest(zad2_matrix)

X, X_approx = zad2.solve(5)

print(X, X_approx)

[-1 -1  1  1 -1] [-1. -1.  1.  1. -1.]
